<a href="https://colab.research.google.com/github/JannaBabicheva/machine-learning-MIPT/blob/main/Babicheva_hw05_bagging_and_oob_bagging_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Simplified BaggingRegressor

In [ ]:
import numpy as np

class SimplifiedBaggingRegressor:
    def __init__(self, num_bags, oob=False):
        self.num_bags = num_bags
        self.oob = oob

    def _generate_splits(self, data: np.ndarray):
        '''
        Generate indices for every bag and store in self.indices_list list.
        '''
        self.indices_list = []
        data_length = len(data)
        for bag in range(self.num_bags):
            # Generate random indices with replacement
            indices = np.random.randint(0, data_length, data_length)
            self.indices_list.append(indices)

    def fit(self, model_constructor, data, target):
        '''
        Fit model on every bag.
        Model constructor with no parameters (and with no ()) is passed to this function.

        example:

        bagging_regressor = SimplifiedBaggingRegressor(num_bags=10, oob=True)
        bagging_regressor.fit(LinearRegression, X, y)
        '''
        self.data = None
        self.target = None
        self._generate_splits(data)
        assert len(set(list(map(len, self.indices_list)))) == 1, 'All bags should be of the same length!'
        assert list(map(len, self.indices_list))[0] == len(data), 'All bags should contain `len(data)` number of elements!'
        self.models_list = []
        for bag in range(self.num_bags):
            model = model_constructor()
            indices = self.indices_list[bag]
            data_bag, target_bag = data[indices], target[indices]
            self.models_list.append(model.fit(data_bag, target_bag))
        if self.oob:
            self.data = data
            self.target = target

    def predict(self, data):
        '''
        Get average prediction for every object from passed dataset
        '''
        predictions = np.zeros((data.shape[0], self.num_bags))
        for i, model in enumerate(self.models_list):
            predictions[:, i] = model.predict(data)
        return np.mean(predictions, axis=1)

    def _get_oob_predictions_from_every_model(self):
        '''
        Generates list of lists, where list i contains predictions for self.data[i] object
        from all models, which have not seen this object during training phase.
        '''
        list_of_predictions_lists = [[] for _ in range(len(self.data))]
        for i, model in enumerate(self.models_list):
            oob_indices = np.setdiff1d(np.arange(len(self.data)), self.indices_list[i])  # Indices not in the bag
            if len(oob_indices) > 0:
                predictions = model.predict(self.data[oob_indices])
                for idx, prediction in zip(oob_indices, predictions):
                    list_of_predictions_lists[idx].append(prediction)

        self.list_of_predictions_lists = np.array(list_of_predictions_lists, dtype=object)

    def _get_averaged_oob_predictions(self):
        '''
        Compute average prediction for every object from training set.
        If object has been used in all bags on training phase, return None instead of prediction.
        '''
        self._get_oob_predictions_from_every_model()
        self.oob_predictions = np.array([
            np.mean(predictions) if len(predictions) > 0 else None
            for predictions in self.list_of_predictions_lists
        ])

    def OOB_score(self):
        '''
        Compute mean square error for all objects, which have at least one prediction.
        '''
        self._get_averaged_oob_predictions()
        errors = []
        for i in range(len(self.oob_predictions)):
            if self.oob_predictions[i] is not None:
                error = (self.oob_predictions[i] - self.target[i])**2
                errors.append(error)
        return np.mean(errors) if errors else None


_generate_splits: Этот метод генерирует случайные индексы для формирования образцов с заменой для каждого пакета.

fit: для каждой модели, которая должна быть подогнана, используются соответствующие индексы для выбора данных и целевых образцов и подгонки модели.

predict: использует средние прогнозы по всем установленным моделям для заданных данных.

_get_oob_predictions_from_every_model и _get_averaged_oob_predictions: Эти методы вычисляют прогнозы "вне пакета" для точек данных, которые не были включены в соответствующие пакеты.

OOB_score: Вычисляет среднеквадратичную ошибку прогнозов, полученных при отсутствии пакета, по сравнению с фактическими целевыми показателями.


Эти реализации дополняют различные функциональные возможности, ожидаемые от базового baggingregressor, такие как загрузка образцов в пакеты, составление прогнозов и вычисление ошибок OOB.

model_constructor, передаваемый в fit, должен быть классом model, способным выполнять методы fit и predict, типичные для интерфейса scikit-learn.